# Proyecto 1 - Visión por Computadora
## Ejercicio 1
## Integrantes:

- Javier Alvarado - 21188
- Mario Guerra - 21008
- Emilio Solano - 21212

In [1]:
import cv2
import mediapipe as mp

# Configuraciones para dibujar
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Si prefieres solo detección de caras, manos, etc., 
# puedes inicializar en su lugar mp.solutions.face_detection o mp.solutions.hands.
mp_holistic = mp.solutions.holistic


In [2]:
def procesar_frame(frame, holistic):
    """
    Procesa un frame de video usando un modelo Holistic de MediaPipe.
    Devuelve el frame con las anotaciones dibujadas.
    """
    
    # Convertimos a RGB para MediaPipe
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Opcional: cambiar la imagen a no-writeable para optimizar
    image_rgb.flags.writeable = False
    
    # Procesa la imagen con el modelo
    results = holistic.process(image_rgb)
    
    # Volver a writeable para dibujar
    image_rgb.flags.writeable = True
    
    # Convertimos de vuelta a BGR (para mostrar con OpenCV)
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    # Dibuja la pose
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image_bgr,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
        )

    # Dibuja los landmarks de la cara
    if results.face_landmarks:
        mp_drawing.draw_landmarks(
            image_bgr,
            results.face_landmarks,
            mp_holistic.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
        )
        mp_drawing.draw_landmarks(
            image_bgr,
            results.face_landmarks,
            mp_holistic.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style()
        )

    # Dibuja los landmarks de la mano izquierda
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            image_bgr,
            results.left_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style(),
            connection_drawing_spec=mp_drawing_styles.get_default_hand_connections_style()
        )

    # Dibuja los landmarks de la mano derecha
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            image_bgr,
            results.right_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style(),
            connection_drawing_spec=mp_drawing_styles.get_default_hand_connections_style()
        )
    
    return image_bgr


In [3]:
# Abre la cámara (índice 0 suele ser la cámara por defecto)
cap = cv2.VideoCapture(0)

# Ajusta el ancho y alto si lo deseas
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

with mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=1, 
    smooth_landmarks=True, 
    enable_segmentation=False, 
    smooth_segmentation=True,
    refine_face_landmarks=True
) as holistic:

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Procesar el frame con la función que definimos
        frame_procesado = procesar_frame(frame, holistic)

        # Mostrar el frame procesado en una ventana
        cv2.imshow("Detección Holistic - MediaPipe", frame_procesado)
        
        # Si se presiona la tecla 'q', salimos
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 